# Heteroscedasticity in Regression

When your regression variance is non-stationary

## A Gaussian Process model for Heteroscedasticity

A common phenomenon when working on continuous regression problems is the non-constant residual variance, also known as heteroscedasticity. Although predicting the mean via MSE-minimisation is often sufficient and more pragmatic, a proper treatment of the variance can be helpful at times.

### Problem definition

At the heart of non-constant variance models, lies the assumption of some functional relation between the input data and the variance of the target variable. Presuming also a Gaussian target variable, we can construct the following probabilistic setup:

$$ y \sim N(m(X), \sigma^{2}(X)) $$

Put plainly, given some input data, the corresponding target should be Gaussian with mean and variance being arbitrary functions of our inputs. Since our today is on the variance, let us simplify things a little with a zero-mean function.

In simple terms, we now don’t expect that a single model would best describe our target function anymore. Instead, a — possibly infinitely large — set of models is considered and our goal is to place a probability distribution (a.k.a. posterior distribution) on this set such that those models that best describe the data (a.k.a. likelihood) given the assumptions we made (a.k.a. prior distributions) are the most likely ones.

This is usually done in weight space by defining our set of models in an implicit manner via the sets of parameters that describe the models’ behaviour — probably the most famous example in Machine Learning are Bayesian Neural Networks. Another, more abstract approach is to directly work in function space, i.e. we now explicitly look for the most likely functions without requiring parameters to describe them in the first place. Since we are working in the Bayesian domain, this also means that prior and posterior distributions aren’t put over parameters anymore but also directly over functions. One of the most iconic frameworks for such modelling is Gaussian Process (GP) regression.

### The Model:

Our goal will be to model the variance of the target variable through a Gaussian Process, which looks as follows:

$$ y \sim N( 0,f^{exp}(X)) $$

$$ f(.) \sim GP(0,k(.,.)+v*\delta_{ij}), f^{exp}(X) = exp(f(X)) $$

This implies that the logarithm of our variance function is a GP — we need to squash the raw GP through an exponential to ensure that the variance will always be greater than zero. Any other function that maps the real line to the positive reals will do here but the exponential arguably the most popular one. The above also implies that the GP is actually a latent component of our model that we only observe indirectly from the data we collect. Finally, we presumed additional noise on the GP kernel via the delta summand which makes the model more stable in practice.

We can derive the posterior distribution derived via Bayes theorem as follows:

$$ p(f|X,y) = N(y|0,f^{exp}(X)) * GP(f|0,k(.,.)+v^{2}*\delta_{ij}) / p(y|X) $$ 

While it is possible to derive the left hand site in closed form for some basic GP models, we cannot do so in our case. Instead we will apply Laplace Approximation and approximate it through a synthetic multivariate normal:

$$ p(f|X,y) \approx N(f| \hat{f},A) $$

In summary, the mean parameter of our approximation should match the mode of the posterior, while it's covariance matrix is the negative inverse of the Hessian matrix of our log-likelihood function:

$$ \hat{f} = argmax_{f} log N(y|0,f^{exp}(X))  + log GP(f|0,k(.,.)+v^{2}*\delta_{ij})$$

$$ A^{-1} = - \nabla^{2} log N(y|0,f^{exp}(X)) $$

To find the approximate mean and optimal kernel hyper-parameters for some example data later on, we will plug in the whole loss into an automatic differentiation package and let the computer do the rest. For the covariance matrix of our approximation, we need to actually calculate the Hessian matrix. A common simplification for GP models is the assumption of independent observations of the target variable given a realisation of the latent GP:

$$ p(y|f) = \prod_{i=1}^{N} p(y_{i}|f) $$

This allows us to simplify the Hessian matrix to be zero everywhere, except for its diagonal which is the second-order derivative of the log-likelihood function with respect to the GP:

$$ H_{ii} = \displaystyle \frac{\partial^{2} log p(y_{i}|f_{i})}{\partial^{2} f_{i}} $$

The right-hand side can be derived by differentiating the standard Gaussian log-likelihood twice with respect to the variance while accounting for our exponential transform:

$$ \displaystyle \frac{\partial^{2} log p(y_{i}|f_{i})}{\partial^{2} f_{i}} = -0.5  \frac{y_{i}^{2}}{exp(f_{i})} $$